# Boilerplate

There's probably a better way than what follows

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio

from nwv2_python_wrapper import *
import nwv2_python_wrapper

# Run

What can we run?

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['EndpointW',
 'PacketW',
 'RequestActionW',
 'TransportInterface',
 'new_transport_interface']

## Transport Setup

Create a new transport interface, asynchronously. This uses the Python `asyncio` event loop, which a Jupyter Notebook already has running.

In [3]:
tiface_fut = new_transport_interface("0.0.0.0", 12346)
tiface_fut

<Future pending cb=[<builtins.PyDoneCallback object at 0x7f3ccf9ab670>()]>

Verify the `Future` has completed.

In [4]:
assert tiface_fut.done()

Call the `result` method to get the result when it's complete.

In [5]:
tiface = tiface_fut.result()
tiface

<(Transport, TransportCmdSend, TransportRspRecv, TransportNotifyRecv)>